In [ ]:
import os
print(os.getcwd())
os.chdir('<YOUR_WORKING_DIR>')
print(os.getcwd())

In [13]:
# pip install -U openai
import openai, os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # This loads variables from .env into os.environ

api_key=os.environ["OPENAI_EMBED_API_KEY"]

In [23]:
endpoint=str(os.environ["OPENAI_EMBED_END_POINT"]) + '/openai/v1/'


In [25]:
# 1) Put your new key in the environment, e.g.:
#   export OPENAI_API_KEY="sk-..."    (macOS/Linux)
#   setx OPENAI_API_KEY "sk-..."      (Windows PowerShell, then restart terminal)
client = OpenAI( base_url=endpoint,api_key=api_key)

MODEL = "o4-mini" #"gpt-4o-mini"  # modern, fast; you can also use "gpt-4o"
# (Choose any currently available model for your account)

def parse_legal_text(text: str) -> str:
    prompt = (
        "Extract and label logical primitives (entities, relationships, "
        "obligations, permissions, prohibitions, conditions) from this legal text:\n"
        f"{text}\n\nReturn as structured data (JSON)."
    )
    resp = client.chat.completions.create(
        model=MODEL,
        # JSON mode helps ensure valid JSON on supported models (e.g., gpt-4o/mini)
        response_format={"type": "json_object"},
        messages=[{"role": "user", "content": prompt}],
    )
    return resp.choices[0].message.content

def augment_retrieval(text: str, retrieval_question: str) -> str:
    prompt = (
        "Given the following legal text, provide relevant statutes, cases, or precedents "
        "that could help answer this question:\n"
        f"{text}\n\nQuestion: {retrieval_question}\n\n"
        "Return a list of recommended sources with brief explanations."
    )
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
    )
    return resp.choices[0].message.content

def analogical_linkage(text: str, precedent_case: str) -> str:
    prompt = (
        f"Compare the reasoning and facts in this legal text:\n{text}\n\n"
        f"with the precedent case:\n{precedent_case}\n\n"
        "Highlight key analogies and differences relevant for legal adjudication."
    )
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
    )
    return resp.choices[0].message.content

def argument_scoring(argument_text: str) -> str:
    prompt = (
        "Score the persuasiveness and legal validity of this argument from 1 (weak) to 5 (strong). "
        "Explain your score.\n\nArgument:\n" + argument_text
    )
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
    )
    return resp.choices[0].message.content

# --- Example usage ---
if __name__ == "__main__":
    legal_text = "No person may bring a vehicle into the park except emergency vehicles responding to an incident."
    retrieval_question = "Are ambulances allowed in the park during emergencies?"
    precedent_case = "Smith v. Town (2010): Firetrucks responding to emergencies were exempt from park access restrictions."
    argument_text = "The law permits emergency vehicles in the park, which should include ambulances during emergencies."

    print("Parsed Primitives:", parse_legal_text(legal_text))
    print("Retrieval Augmentation:", augment_retrieval(legal_text, retrieval_question))
    print("Analogical Linkage:", analogical_linkage(legal_text, precedent_case))
    print("Argument Score:", argument_scoring(argument_text))


Parsed Primitives: {
  "entities": [
    { "id": "e1", "name": "person",         "type": "class"     },
    { "id": "e2", "name": "vehicle",        "type": "class"     },
    { "id": "e3", "name": "park",           "type": "location"  },
    { "id": "e4", "name": "emergency vehicle", "type": "class"  },
    { "id": "e5", "name": "incident",       "type": "event"     }
  ],
  "relationships": [
    {
      "id": "r1",
      "name": "bring_into",
      "arguments": ["actor", "object", "location"],
      "description": "Actor brings an object into a location"
    },
    {
      "id": "r2",
      "name": "respond_to",
      "arguments": ["actor", "event"],
      "description": "Actor is responding to an event"
    }
  ],
  "obligations": [],
  "prohibitions": [
    {
      "id": "p1",
      "description": "No person may bring a vehicle into the park.",
      "actor": "person",
      "action": {
        "predicate": "bring_into",
        "object": "vehicle",
        "location": "park"
     